In [97]:
import numpy as np
import pandas as pd
import plotly.express as px

## 1.1 EU Data

In [99]:
df_eu = pd.read_csv('Data/EU_MILK_trade_data_en.csv')
df_eu.head()

,Flow,Marketing Year,Month,Month Order in MY,Month Date,Member State,Partner,Product Group,Product Code (CN),Quantity in tonnes,Value in thousand euro
0,EXPORT,2022,Oct,10,01/10/2022,Austria,Albania,Cheese,4063039,17.677,86.192
1,EXPORT,2022,Oct,10,01/10/2022,Austria,Angola,Cheese,4061050,0.472,2.189
2,EXPORT,2022,Oct,10,01/10/2022,Austria,Angola,Cheese,4063031,9.862,41.178
3,EXPORT,2022,Oct,10,01/10/2022,Austria,Angola,Cheese,4063039,3.177,14.467
4,EXPORT,2022,Oct,10,01/10/2022,Austria,Antigua and Barbuda,Cheese,4063031,1.800,9.212


In [100]:
df_eu = df_eu[(df_eu['Product Code (CN)'].astype('string').str[:5]=='40510') & (df_eu['Flow']=='EXPORT')]

In [101]:
df_eu.isna().sum()

Flow                      0
Marketing Year            0
Month                     0
Month Order in MY         0
Month Date                0
Member State              0
Partner                   0
Product Group             0
Product Code (CN)         0
Quantity in tonnes        0
Value in thousand euro    0
dtype: int64

In [102]:
df_eu['Product Group'].unique()

array(['Butter', 'Butteroil', 'Dairy spreads'], dtype=object)

In [ ]:
# Use lambda function to convert the Month Date values to datetime
df_eu['Month Date'] = df_eu['Month Date'].map(lambda x: datetime.strptime(x, '%d/%m/%Y'))


In [103]:
df_eu['Month Date'] = df_eu['Month Date'].map(make_timestamp)

In [104]:
df_eu

,Flow,Marketing Year,Month,Month Order in MY,Month Date,Member State,Partner,Product Group,Product Code (CN),Quantity in tonnes,Value in thousand euro
35,EXPORT,2022,Oct,10,2022-10-01,Austria,Holy See (Vatican City State),Butter,4051011,1.800,14.275
89,EXPORT,2022,Oct,10,2022-10-01,Austria,Switzerland,Butter,4051011,0.010,0.084
90,EXPORT,2022,Oct,10,2022-10-01,Austria,Switzerland,Butteroil,4051090,0.000,0.007
91,EXPORT,2022,Oct,10,2022-10-01,Austria,Switzerland,Butteroil,4059090,0.005,0.343
130,EXPORT,2022,Oct,10,2022-10-01,Austria,United Arab Emirates,Butter,4051011,0.376,1.564
...,...,...,...,...,...,...,...,...,...,...,...
1465665,EXPORT,2010,Jan,1,2010-01-01,United Kingdom,St Vincent and the Grenadines,Butter,4051011,1.550,5.475
1465678,EXPORT,2010,Jan,1,2010-01-01,United Kingdom,Thailand,Butteroil,4059090,0.120,1.135
1465684,EXPORT,2010,Jan,1,2010-01-01,United Kingdom,United Arab Emirates,Butteroil,4059090,0.715,3.401
1465702,EXPORT,2010,Jan,1,2010-01-01,United Kingdom,United States of America,Butter,4051011,7.845,35.826


In [105]:
df_eu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98808 entries, 35 to 1465703
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Flow                    98808 non-null  object        
 1   Marketing Year          98808 non-null  int64         
 2   Month                   98808 non-null  object        
 3   Month Order in MY       98808 non-null  int64         
 4   Month Date              98808 non-null  datetime64[ns]
 5   Member State            98808 non-null  object        
 6   Partner                 98808 non-null  object        
 7   Product Group           98808 non-null  object        
 8   Product Code (CN)       98808 non-null  int64         
 9   Quantity in tonnes      98808 non-null  float64       
 10  Value in thousand euro  98808 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(5)
memory usage: 9.0+ MB


## 1.2 New Zealand Data

In [106]:
df_nz = pd.read_csv('Data/EXP326301_20221220_044321_37.csv',skiprows=1)

In [107]:
df_nz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       175 non-null    object 
 1   Dairy Produce: Butter (HS 0405)  154 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.0+ KB


In [108]:
df_nz.rename(columns = {'Unnamed: 0':'Month Date'}, inplace = True)

In [109]:
df_nz.tail(30)

,Month Date,Dairy Produce: Butter (HS 0405)
153,2022M10,42.0
154,Table information:,NaN
155,Units:,NaN
156,"Tonnes, Magnitude = Thousands",NaN
157,NaN,NaN
158,Footnotes:,NaN
159,,NaN
160,Symbols:,NaN
161,.. figure not available,NaN
162,C: Confidential,NaN


In [110]:
df_nz = df_nz.iloc[:-29]
df_nz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Month Date                       154 non-null    object 
 1   Dairy Produce: Butter (HS 0405)  154 non-null    float64
dtypes: float64(1), object(1)
memory usage: 2.5+ KB


In [111]:
df_nz['Quantity in tonnes'] = df_nz['Dairy Produce: Butter (HS 0405)'] * 1000
df_nz.drop(['Dairy Produce: Butter (HS 0405)'],axis=1,inplace=True)
df_nz.tail(10)

,Month Date,Quantity in tonnes
144,2022M01,38000.0
145,2022M02,37000.0
146,2022M03,43000.0
147,2022M04,34000.0
148,2022M05,36000.0
149,2022M06,35000.0
150,2022M07,46000.0
151,2022M08,19000.0
152,2022M09,31000.0
153,2022M10,42000.0


In [112]:
df_nz['Month Date'] = df_nz['Month Date'].map(make_timestamp)
df_nz.isna().sum()

Month Date            0
Quantity in tonnes    0
dtype: int64

In [113]:
df_nz

,Month Date,Quantity in tonnes
0,2010-01-01,43000.0
1,2010-02-01,30000.0
2,2010-03-01,35000.0
3,2010-04-01,33000.0
4,2010-05-01,31000.0
...,...,...
149,2022-06-01,35000.0
150,2022-07-01,46000.0
151,2022-08-01,19000.0
152,2022-09-01,31000.0


## 1.3 NewZealand Data 2

In [236]:
filename = 'Data/NZ/Trade_Map_-_List_of_importing_markets_for_a_product_exported_by_New_Zealand-Quantity.txt'
df_nz_x_value = pd.read_csv(filename, sep='\t', lineterminator='\r')

filename = 'Data/NZ/Trade_Map_-_List_of_importing_markets_for_a_product_exported_by_New_Zealand-Value.txt'
df_nz_x_quantity = pd.read_csv(filename, sep='\t', lineterminator='\r')

filename = 'Data/NZ/Trade_Map_-_List_of_supplying_markets_for_a_product_imported_by_New_Zealand-Quantity.txt'
df_nz_i_value = pd.read_csv(filename, sep='\t', lineterminator='\r')

filename = 'Data/NZ/Trade_Map_-_List_of_supplying_markets_for_a_product_imported_by_New_Zealand-Value.txt'
df_nz_i_quantity = pd.read_csv(filename, sep='\t', lineterminator='\r')

In [238]:
df_nz_i_quantity

,Exporters,Imported value in 2021-M10,Imported value in 2021-M11,Imported value in 2021-M12,Imported value in 2022-M01,Imported value in 2022-M02,Imported value in 2022-M03,Imported value in 2022-M04,Imported value in 2022-M05,Imported value in 2022-M06,Imported value in 2022-M07,Imported value in 2022-M08,Imported value in 2022-M09,Unnamed: 13
0,"\n""World""",175.0,226.0,314.0,113.0,365.0,156.0,296.0,897.0,533.0,184.0,1074.0,648.0,NaN
1,"\n""New Zealand""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,674.0,0.0,0.0,260.0,544.0,NaN
2,"\n""Australia""",7.0,117.0,29.0,32.0,32.0,34.0,26.0,7.0,40.0,0.0,7.0,53.0,NaN
3,"\n""Denmark""",114.0,0.0,90.0,80.0,25.0,93.0,121.0,73.0,52.0,48.0,0.0,47.0,NaN
4,"\n""India""",46.0,107.0,80.0,0.0,307.0,28.0,1.0,141.0,146.0,137.0,319.0,4.0,NaN
5,"\n""United States of America""",7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
6,"\n""Fiji""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,NaN
7,"\n""France""",0.0,0.0,114.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
8,"\n""Ireland""",0.0,0.0,0.0,0.0,0.0,0.0,146.0,0.0,293.0,0.0,481.0,0.0,NaN
9,"\n""Lithuania""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [220]:
# Use lambda function to strip unwanted characters from beginning (lstrip) and end (rstrip) of the Importers values
df_nz_x_quantity['Importers'] = df_nz_x_quantity['Importers'].map(lambda x: x.lstrip('\n"').rstrip('"'))


In [221]:
df_nz_x_quantity

,Importers,Exported value in 2010-M01,Exported value in 2010-M02,Exported value in 2010-M03,Exported value in 2010-M04,Exported value in 2010-M05,Exported value in 2010-M06,Exported value in 2010-M07,Exported value in 2010-M08,Exported value in 2010-M09,...,Exported value in 2010-M12,Exported value in 2011-M01,Exported value in 2011-M02,Exported value in 2011-M03,Exported value in 2011-M04,Exported value in 2011-M05,Exported value in 2011-M06,Exported value in 2011-M07,Exported value in 2011-M08,Unnamed: 21
0,World,692494.0,535837.0,687953.0,645822.0,644285.0,560627.0,528068.0,362827.0,497635.0,...,776429.0,802555.0,749642.0,910707.0,944726.0,791099.0,732157.0,690368.0,480259.0,NaN
1,China,120237.0,82648.0,115076.0,122537.0,125422.0,93945.0,83273.0,30109.0,96066.0,...,177618.0,176497.0,223495.0,213993.0,155856.0,132221.0,86493.0,86946.0,45733.0,NaN
2,Australia,29028.0,36001.0,34483.0,24127.0,32262.0,27680.0,30488.0,35070.0,34373.0,...,23359.0,34296.0,29497.0,29271.0,30327.0,38077.0,33482.0,29458.0,32819.0,NaN
3,Philippines,46032.0,29131.0,37400.0,38139.0,28334.0,27527.0,34034.0,13128.0,22385.0,...,39710.0,31804.0,30494.0,26249.0,42921.0,30080.0,42365.0,32312.0,27715.0,NaN
4,Sri Lanka,15324.0,16166.0,20476.0,15653.0,16093.0,22149.0,14032.0,17411.0,14843.0,...,13831.0,26327.0,15246.0,20654.0,18016.0,20155.0,23552.0,17850.0,25963.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,Yemen,6373.0,5233.0,6634.0,1175.0,3810.0,184.0,999.0,687.0,3319.0,...,408.0,1786.0,309.0,2509.0,879.0,1707.0,0.0,0.0,0.0,NaN
152,Zambia,0.0,246.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,268.0,0.0,0.0,0.0,NaN
153,Burkina Faso,0.0,92.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
154,Uruguay,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [222]:
df_nz_x_quantity = df_nz_x_quantity[(df_nz_x_quantity['Importers'] != 'World')]
df_nz_x_quantity = df_nz_x_quantity.iloc[:-1]
df_nz_x_quantity = df_nz_x_quantity.iloc[: , :-1]
df_nz_x_quantity

,Importers,Exported value in 2010-M01,Exported value in 2010-M02,Exported value in 2010-M03,Exported value in 2010-M04,Exported value in 2010-M05,Exported value in 2010-M06,Exported value in 2010-M07,Exported value in 2010-M08,Exported value in 2010-M09,...,Exported value in 2010-M11,Exported value in 2010-M12,Exported value in 2011-M01,Exported value in 2011-M02,Exported value in 2011-M03,Exported value in 2011-M04,Exported value in 2011-M05,Exported value in 2011-M06,Exported value in 2011-M07,Exported value in 2011-M08
1,China,120237.0,82648.0,115076.0,122537.0,125422.0,93945.0,83273.0,30109.0,96066.0,...,126005.0,177618.0,176497.0,223495.0,213993.0,155856.0,132221.0,86493.0,86946.0,45733.0
2,Australia,29028.0,36001.0,34483.0,24127.0,32262.0,27680.0,30488.0,35070.0,34373.0,...,31393.0,23359.0,34296.0,29497.0,29271.0,30327.0,38077.0,33482.0,29458.0,32819.0
3,Philippines,46032.0,29131.0,37400.0,38139.0,28334.0,27527.0,34034.0,13128.0,22385.0,...,29963.0,39710.0,31804.0,30494.0,26249.0,42921.0,30080.0,42365.0,32312.0,27715.0
4,Sri Lanka,15324.0,16166.0,20476.0,15653.0,16093.0,22149.0,14032.0,17411.0,14843.0,...,18425.0,13831.0,26327.0,15246.0,20654.0,18016.0,20155.0,23552.0,17850.0,25963.0
5,Japan,28957.0,22866.0,20126.0,23576.0,26812.0,21467.0,20792.0,16853.0,16404.0,...,24666.0,22554.0,24744.0,30931.0,24383.0,29132.0,25134.0,26487.0,39815.0,24379.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,Israel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0
151,Yemen,6373.0,5233.0,6634.0,1175.0,3810.0,184.0,999.0,687.0,3319.0,...,5893.0,408.0,1786.0,309.0,2509.0,879.0,1707.0,0.0,0.0,0.0
152,Zambia,0.0,246.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,268.0,0.0,0.0,0.0
153,Burkina Faso,0.0,92.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [223]:
v_name = 'Quantity in tonnes'
df_nz_x_quantity = df_nz_x_quantity.melt(id_vars=['Importers'], var_name='Month Date', value_name=v_name)

In [224]:
flow = 'EXPORT'
df_nz_x_quantity['Flow'] = flow

In [ ]:
df_nz_x_quantity.rename(columns = {'Imoprters':'Partner'}, inplace = True)

In [226]:
df_nz_x_quantity

,Importers,Month Date,Quantity in tonnes,Flow
0,China,Exported value in 2010-M01,120237.0,EXPORT
1,Australia,Exported value in 2010-M01,29028.0,EXPORT
2,Philippines,Exported value in 2010-M01,46032.0,EXPORT
3,Sri Lanka,Exported value in 2010-M01,15324.0,EXPORT
4,Japan,Exported value in 2010-M01,28957.0,EXPORT
...,...,...,...,...
3075,Israel,Exported value in 2011-M08,0.0,EXPORT
3076,Yemen,Exported value in 2011-M08,0.0,EXPORT
3077,Zambia,Exported value in 2011-M08,0.0,EXPORT
3078,Burkina Faso,Exported value in 2011-M08,0.0,EXPORT


In [227]:
# Use lambda function to strip unwanted text at the beginning of  
# the Month Date values and convert them to datetime
df_nz_x_quantity['Month Date'] = df_nz_x_quantity['Month Date'].map(lambda x: datetime.strptime(x.lstrip('Exported value in '), '%Y-M%m'))


In [229]:
df_nz_x_quantity.isna().sum()

Importers             0
Month Date            0
Quantity in tonnes    0
Flow                  0
dtype: int64

In [202]:
df_nz = df_nz_x_quantity

## 2.1 Plot

In [115]:
df_plot = df_eu.copy() # Use deep copy to preserve the contents of df_eu 

In [116]:
df_plot.drop(
    ['Flow','Marketing Year','Month','Month Order in MY','Partner','Product Group','Product Code (CN)'],
    axis=1,
    inplace=True)


In [117]:
df_plot

,Month Date,Member State,Quantity in tonnes,Value in thousand euro
35,2022-10-01,Austria,1.800,14.275
89,2022-10-01,Austria,0.010,0.084
90,2022-10-01,Austria,0.000,0.007
91,2022-10-01,Austria,0.005,0.343
130,2022-10-01,Austria,0.376,1.564
...,...,...,...,...
1465665,2010-01-01,United Kingdom,1.550,5.475
1465678,2010-01-01,United Kingdom,0.120,1.135
1465684,2010-01-01,United Kingdom,0.715,3.401
1465702,2010-01-01,United Kingdom,7.845,35.826


In [118]:
df_plot = df_plot.groupby(['Member State','Month Date']).sum().reset_index()

In [119]:
df_plot

,Member State,Month Date,Quantity in tonnes,Value in thousand euro
0,Austria,2010-01-01,29.582,115.872
1,Austria,2010-02-01,28.778,124.012
2,Austria,2010-03-01,24.965,111.098
3,Austria,2010-04-01,13.012,53.326
4,Austria,2010-05-01,25.808,109.539
...,...,...,...,...
3707,United Kingdom,2019-09-01,928.317,3953.774
3708,United Kingdom,2019-10-01,1428.647,5978.192
3709,United Kingdom,2019-11-01,1379.350,5853.372
3710,United Kingdom,2019-12-01,737.340,3124.072


In [120]:
df_nz

,Month Date,Quantity in tonnes
0,2010-01-01,43000.0
1,2010-02-01,30000.0
2,2010-03-01,35000.0
3,2010-04-01,33000.0
4,2010-05-01,31000.0
...,...,...
149,2022-06-01,35000.0
150,2022-07-01,46000.0
151,2022-08-01,19000.0
152,2022-09-01,31000.0


In [121]:
df_nz['Member State'] = 'New Zealand'
df_nz['Value in thousand euro'] = 0

In [122]:
df_nz

,Month Date,Quantity in tonnes,Member State,Value in thousand euro
0,2010-01-01,43000.0,New Zealand,0
1,2010-02-01,30000.0,New Zealand,0
2,2010-03-01,35000.0,New Zealand,0
3,2010-04-01,33000.0,New Zealand,0
4,2010-05-01,31000.0,New Zealand,0
...,...,...,...,...
149,2022-06-01,35000.0,New Zealand,0
150,2022-07-01,46000.0,New Zealand,0
151,2022-08-01,19000.0,New Zealand,0
152,2022-09-01,31000.0,New Zealand,0


### Concatenate both sets together

In [123]:
df_plot = pd.concat([df_plot,df_nz],ignore_index=True)

In [124]:
df_plot = df_plot.groupby([pd.Grouper(key='Month Date',freq='1Y'),'Member State']).sum().reset_index()

In [125]:
df_plot

,Month Date,Member State,Quantity in tonnes,Value in thousand euro
0,2010-12-31,Austria,371.830,1778.492
1,2010-12-31,Belgium,16590.600,65219.257
2,2010-12-31,Bulgaria,80.775,221.054
3,2010-12-31,Cyprus,28.336,72.466
4,2010-12-31,Czechia,884.249,3132.395
...,...,...,...,...
355,2022-12-31,Romania,189.247,1270.677
356,2022-12-31,Slovakia,58.698,412.464
357,2022-12-31,Slovenia,118.344,758.573
358,2022-12-31,Spain,1335.926,9834.860


In [126]:
fig = px.line(df_plot, x="Month Date", y="Quantity in tonnes", color='Member State', title='Butter Exports')
fig.show()

In [20]:
df_plot

,Flow,Marketing Year,Month,Month Order in MY,Month Date,Member State,Partner,Product Group,Product Code (CN),Quantity in tonnes,Value in thousand euro
35,EXPORT,2022,Oct,10,2022-10-01,Austria,Holy See (Vatican City State),Butter,4051011,1.800,14.275
89,EXPORT,2022,Oct,10,2022-10-01,Austria,Switzerland,Butter,4051011,0.010,0.084
90,EXPORT,2022,Oct,10,2022-10-01,Austria,Switzerland,Butteroil,4051090,0.000,0.007
91,EXPORT,2022,Oct,10,2022-10-01,Austria,Switzerland,Butteroil,4059090,0.005,0.343
130,EXPORT,2022,Oct,10,2022-10-01,Austria,United Arab Emirates,Butter,4051011,0.376,1.564
...,...,...,...,...,...,...,...,...,...,...,...
1465665,EXPORT,2010,Jan,1,2010-01-01,United Kingdom,St Vincent and the Grenadines,Butter,4051011,1.550,5.475
1465678,EXPORT,2010,Jan,1,2010-01-01,United Kingdom,Thailand,Butteroil,4059090,0.120,1.135
1465684,EXPORT,2010,Jan,1,2010-01-01,United Kingdom,United Arab Emirates,Butteroil,4059090,0.715,3.401
1465702,EXPORT,2010,Jan,1,2010-01-01,United Kingdom,United States of America,Butter,4051011,7.845,35.826
